# Subset barcode runs
This Python Jupyter notebook is creates a copy of the barcode runs file that contains just specific samples.

First, import Python modules:

In [1]:
import os

import pandas as pd

import yaml

Read in the "master" barcode runs CSV file and the runs used to compute the escape scores for each sample:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)

barcode_runs_file = os.path.join('../', config['barcode_runs'])
print(f"Reading barcode runs from {barcode_runs_file}")
barcode_runs = pd.read_csv(barcode_runs_file)

escape_score_samples_file = os.path.join('../', config['escape_score_samples'])
print(f"Reading escape score samples from {escape_score_samples_file}")
escape_score_samples = pd.read_csv(escape_score_samples_file)

Reading barcode runs from ../data/barcode_runs.csv
Reading escape score samples from ../results/escape_scores/samples.csv


Now read in the samples to subset:

In [3]:
samples_to_subset_df = pd.read_csv('samples_to_subset.csv')
print('Here are the samples we will subset to:')
display(samples_to_subset_df)

samples_to_subset = samples_to_subset_df['sample'].tolist()
assert len(samples_to_subset) == len(set(samples_to_subset)), 'duplicate samples to subset'

Here are the samples we will subset to:


,sample
0,12C_d152_80
1,12C_d61_160
2,13_d121_1250
3,13_d15_200
4,1C_d113_200
...,...
76,M15-day-119_200
77,M20-day-119_200
78,M21-day-119_200
79,M22-day-119_200


Now get all the escape-score samples of interest:

In [4]:
if not set(samples_to_subset).issubset(escape_score_samples['name']):
    raise ValueError(f"Not all samples to subset are in the escape score samples.")
    
samples_subset = (
    escape_score_samples
    .query('name in @samples_to_subset')
    .reset_index(drop=True)
    )

print(f"Here are the samples for which we are subsetting barcode runs:")
display(samples_subset)

Here are the samples for which we are subsetting barcode runs:


,name,library,antibody,concentration,concentration_units,date,pre_sample,post_sample,frac_escape,pre_cells_sorted,post_cells_sorted
0,12C_d152_80,lib1,12C_d152,80,dilution,200910,expt_34-41-none-0-reference,expt_40-12C_d152-80-escape,0.040,530574.0,160000000.0
1,12C_d152_80,lib2,12C_d152,80,dilution,200910,expt_34-41-none-0-reference,expt_40-12C_d152-80-escape,0.052,528440.0,160000000.0
2,12C_d61_160,lib1,12C_d61,160,dilution,200910,expt_34-41-none-0-reference,expt_37-12C_d61-160-escape,0.033,289524.0,160000000.0
3,12C_d61_160,lib2,12C_d61,160,dilution,200910,expt_34-41-none-0-reference,expt_37-12C_d61-160-escape,0.031,373381.0,160000000.0
4,13_d121_1250,lib1,13_d121,1250,dilution,200917,expt_46-52-none-0-reference,expt_52-13_d121-1250-escape,0.042,521978.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
157,REGN10933_400,lib2,REGN10933,400,ng_per_mL,200904,expt_24-33-none-0-reference,expt_24-REGN10933-400-escape,0.154,984237.0,160000000.0
158,REGN10933+REGN10987_400,lib1,REGN10933+REGN10987,400,ng_per_mL,200904,expt_24-33-none-0-reference,expt_26-REGN10933+REGN10987-400-escape,0.101,635898.0,160000000.0
159,REGN10933+REGN10987_400,lib2,REGN10933+REGN10987,400,ng_per_mL,200904,expt_24-33-none-0-reference,expt_26-REGN10933+REGN10987-400-escape,0.083,585188.0,160000000.0
160,REGN10987_400,lib1,REGN10987,400,ng_per_mL,200904,expt_24-33-none-0-reference,expt_25-REGN10987-400-escape,0.148,1155004.0,160000000.0


Now unfold the samples of interest into the actual relevant barcode runs (this requires getting both the pre- and post-selection run for each sample):

In [5]:
barcode_runs_subset = (
    samples_subset
    .melt(id_vars=['name', 'library'],
          value_vars=['pre_sample', 'post_sample'],
          value_name='sample',
          var_name='sample_type')
    [['library', 'sample']]
    .drop_duplicates()
    .reset_index(drop=True)
    .merge(barcode_runs,
           how='left',
           on=['library', 'sample'],
           validate='one_to_one',
           )
    )

assert barcode_runs_subset['R1'].notnull().all(), 'some barcode runs missing R1'

barcode_runs_subset_file = 'barcode_runs_subset.csv'
print(f"Here are the subsetted barcode runs. Writing to {barcode_runs_subset_file}")
barcode_runs_subset.to_csv(barcode_runs_subset_file, index=False)
display(barcode_runs_subset)

Here are the subsetted barcode runs. Writing to barcode_runs_subset.csv


,library,sample,date,experiment,antibody,concentration,concentration_units,group,selection,frac_escape,cells_sorted,R1
0,lib1,expt_34-41-none-0-reference,200910,expt_34-41,none,0,dilution,clinical_serum,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200914_D00300_10...
1,lib2,expt_34-41-none-0-reference,200910,expt_34-41,none,0,dilution,clinical_serum,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200914_D00300_10...
2,lib1,expt_46-52-none-0-reference,200917,expt_46-52,none,0,dilution,clinical_serum,reference,NaN,NaN,/shared/ngs/illumina/agreaney/201012_D00300_10...
3,lib2,expt_46-52-none-0-reference,200917,expt_46-52,none,0,dilution,clinical_serum,reference,NaN,NaN,/shared/ngs/illumina/agreaney/201012_D00300_10...
4,lib1,expt_24-33-none-0-reference,200904,expt_24-33,none,0,ng_per_mL,clinical_serum,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200914_D00300_10...
...,...,...,...,...,...,...,...,...,...,...,...,...
187,lib2,expt_24-REGN10933-400-escape,200904,expt_24,REGN10933,400,ng_per_mL,clinical_serum,escape,0.154,984237.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
188,lib1,expt_26-REGN10933+REGN10987-400-escape,200904,expt_26,REGN10933+REGN10987,400,ng_per_mL,clinical_serum,escape,0.101,635898.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
189,lib2,expt_26-REGN10933+REGN10987-400-escape,200904,expt_26,REGN10933+REGN10987,400,ng_per_mL,clinical_serum,escape,0.083,585188.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
190,lib1,expt_25-REGN10987-400-escape,200904,expt_25,REGN10987,400,ng_per_mL,clinical_serum,escape,0.148,1155004.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
